In [1]:
import pandas as pd    # to load dataset
import numpy as np     # for mathematic equation
from nltk.corpus import stopwords   # to get collection of stopwords
from sklearn.model_selection import train_test_split       # for splitting dataset
from tensorflow.keras.preprocessing.text import Tokenizer  # to encode text to int
from tensorflow.keras.preprocessing.sequence import pad_sequences   # to do padding or truncating
from tensorflow.keras.models import Sequential     # the model
from tensorflow.keras.layers import Embedding, LSTM, Dense # layers of the architecture
from tensorflow.keras.callbacks import ModelCheckpoint   # save model
from tensorflow.keras.models import load_model   # load saved model
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import re
import tensorflow as tf

/opt/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:17: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, defaultdict


In [2]:
data = pd.read_csv( '/Volumes/Cisco/Fall2021/onnx-exchange/Training/IMDB Dataset.csv')
print(data)

                                                  review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]


In [3]:
english_stops = set(stopwords.words('english'))

In [4]:
def load_dataset():
    df = pd.read_csv('/Volumes/Cisco/Fall2021/onnx-exchange/Training/IMDB Dataset.csv')
    x_data = df['review']       # Reviews/Input
    y_data = df['sentiment']    # Sentiment/Output

    # PRE-PROCESS REVIEW
    x_data = x_data.replace({'<.*?>': ''}, regex = True)          # remove html tag
    x_data = x_data.replace({'[^A-Za-z]': ' '}, regex = True)     # remove non alphabet
    x_data = x_data.apply(lambda review: [w for w in review.split() if w not in english_stops])  # remove stop words
    x_data = x_data.apply(lambda review: [w.lower() for w in review])   # lower case
    
    # ENCODE SENTIMENT -> 0 & 1
    y_data = y_data.replace('positive', 1)
    y_data = y_data.replace('negative', 0)

    return x_data, y_data

x_data, y_data = load_dataset()

print('Reviews')
print(x_data, '\n')
print('Sentiment')
print(y_data)

Reviews
0        [one, reviewers, mentioned, watching, oz, epis...
1        [a, wonderful, little, production, the, filmin...
2        [i, thought, wonderful, way, spend, time, hot,...
3        [basically, family, little, boy, jake, thinks,...
4        [petter, mattei, love, time, money, visually, ...
                               ...                        
49995    [i, thought, movie, right, good, job, it, crea...
49996    [bad, plot, bad, dialogue, bad, acting, idioti...
49997    [i, catholic, taught, parochial, elementary, s...
49998    [i, going, disagree, previous, comment, side, ...
49999    [no, one, expects, star, trek, movies, high, a...
Name: review, Length: 50000, dtype: object 

Sentiment
0        1
1        1
2        1
3        0
4        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: sentiment, Length: 50000, dtype: int64


In [5]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2)

print('Train Set')
print(x_train, '\n')
print(x_test, '\n')
print('Test Set')
print(y_train, '\n')
print(y_test)

Train Set
46019    [as, rule, full, moon, production, logo, warni...
33405    [oh, boy, films, like, really, bother, if, mov...
20128    [one, favourite, domestic, movies, i, know, pe...
42259    [aka, aspect, ratio, x, within, frame, triptyc...
803      [it, getting, want, want, and, message, bluebe...
                               ...                        
12995    [this, hodge, podge, adapted, gore, vidal, nov...
20995    [i, purchased, film, dvd, waste, film, bad, th...
24594    [this, sorry, excuse, film, reminded, great, d...
24966    [mr, baseball, film, paradoxes, written, filme...
18213    [full, marks, pacino, rendering, speech, dead,...
Name: review, Length: 40000, dtype: object 

48123    [i, really, enjoyed, first, half, hour, movie,...
44939    [it, complete, nonsense, i, studied, nazism, r...
3182     [my, girlfriend, i, stunned, bad, film, after,...
1797     [i, give, movie, two, stars, teems, beautiful,...
3608     [i, seen, movie, many, times, recently, read, ...
 

In [6]:
def get_max_length():
    review_length = []
    for review in x_train:
        review_length.append(len(review))

    return int(np.ceil(np.mean(review_length)))

In [7]:
# ENCODE REVIEW
token = Tokenizer(lower=False)    # no need lower, because already lowered the data in load_data()
token.fit_on_texts(x_train)
x_train = token.texts_to_sequences(x_train)
x_test = token.texts_to_sequences(x_test)

max_length = get_max_length()

x_train = pad_sequences(x_train, maxlen=max_length, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=max_length, padding='post', truncating='post')

total_words = len(token.word_index) + 1   # add 1 because of 0 padding

print('Encoded X Train\n', x_train, '\n')
print('Encoded X Test\n', x_test, '\n')
print('Maximum review length: ', max_length)

Encoded X Train
 [[ 109 2368  267 ...    0    0    0]
 [ 324  300   35 ...    0    0    0]
 [   5 1572 5490 ...    0    0    0]
 ...
 [   8  669 1281 ... 9548 4882 9548]
 [ 345 2619    4 ... 1397  594  902]
 [ 267 3866 2908 ...    0    0    0]] 

Encoded X Test
 [[    1    13   426 ...     0     0     0]
 [    7   504  1826 ...     0     0     0]
 [  215   824     1 ...     0     0     0]
 ...
 [  424   329     6 ...     0     0     0]
 [   49 10095  5685 ...    60     1   195]
 [  159     5    13 ...     0     0     0]] 

Maximum review length:  131


In [8]:
training_id = 6
model_short_name = 'lstm'
framework = 'keras'

In [9]:
import time
import os
import copy
import csv
import pandas as pd
from datetime import datetime

date = datetime.today().strftime('%Y-%m-%d')

In [10]:
path = '/Volumes/Cisco/Fall2021/onnx-exchange/Training/Keras/lstm/'
since_0 = time.time()
#model_path = 'tf_Lenet5_mnist_2021-08-24-10:35:35'
model_name = 'tf_lstm-imdb_2021-10-31_{}'.format(training_id)
model = tf.keras.models.load_model(path+ model_name+'.h5')
t_elapsed_0 = time.time() - since_0
size0 = os.path.getsize(path+ model_name+'.h5')
size0

144511984

In [11]:
size1 = os.path.getsize(path+ model_name+'.h5')
size1

size2 = os.path.getsize(path+ model_name+'.h5')
print(size1, size2)

144511984 144511984


In [10]:
from torch.utils.data import TensorDataset, DataLoader
import torch

In [11]:
import onnx
import onnxruntime
import coremltools
import time
import tf2onnx

Using TensorFlow backend.


In [12]:
onnx_path = '/Volumes/Cisco/Fall2021/onnx-exchange/conversion/onnx/'
coreml_path = '/Volumes/Cisco/Fall2021/onnx-exchange/conversion/coremltools/'
error_path = '/Volumes/Cisco/Fall2021/onnx-exchange/conversion/errors/'

In [15]:
#x_prev = 
def to_onnx(i, x, data_writer_error, data_writer_error2, data_writer_run, batch_size):
    # Input to the model
    #device_reset = cuda.get_current_device()
    #device_reset.reset()
    #x.cuda()
   
    print("converting for batch: ", i)
    
    #torch.random.manual_seed(42)
    #x = torch.randn(10000, 3, 32, 32, requires_grad=True)
    since_ = time.time()
    since_1 = time.time()
    #model = torch.load(path+model_name+'.pth')
    try:
        with tf.device('/cpu:0'): 
            k_predict = model.predict(x)
        t_elapsed_1 = time.time() - since_1
        # Export the model

        if not os.path.exists(onnx_path+framework+"/{}".format(model_short_name)):
            os.makedirs(onnx_path+framework+"/{}".format(model_short_name))
        if not os.path.exists(coreml_path+framework+"/{}".format(model_short_name)):
            os.makedirs(coreml_path+framework+"/{}".format(model_short_name))

        since_1 = time.time()

        since_onnx = time.time()
        model_proto, external_tensor_storage = tf2onnx.convert.from_keras(model,
                    input_signature=None, opset=11, custom_ops=None,
                    custom_op_handlers=None, custom_rewriter=None,
                    inputs_as_nchw=None, extra_opset=None, shape_override=None,
                     target=None, large_model=False, output_path=onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))

        t_elapsed_2 = time.time() - since_1


        onnx_model = onnx.load(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
        onnx.checker.check_model(onnx_model)
        size2 = os.path.getsize(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
        #def to_numpy(tensor):
        #    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()
        ort_session = onnxruntime.InferenceSession(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
        since_1 = time.time()
        ort_inputs = {ort_session.get_inputs()[0].name: x}
        ort_outs = ort_session.run(None, ort_inputs)
        t_elapsed_3 = time.time() - since_1
        t_elapsed_ = time.time() - since_
        t_elapsed_onnx = time.time() - since_onnx
        # compare ONNX Runtime and PyTorch results
        print("\n*********\n\n")
        #time_diff = t_elapsed_0+t_elapsed_1, t_elapsed_2, t_elapsed_3

        abs_err = np.absolute(k_predict-ort_outs[0])
        rel_err = np.absolute(k_predict-ort_outs[0])/ np.absolute(ort_outs[0])
        #print('Batch: ', i, abs_err, rel_err)



        ## Converting the coremltool
        since_1 = time.time()
        coreml_model = coremltools.convert(model)
        t_elapsed_4 = time.time() - since_1
        since_1 = time.time()
        coreml_model.save(coreml_path+framework+'/{}/{}.mlmodel'.format(model_short_name, model_name))
        t_elapsed_5 = time.time() - since_1

        split_ = str(coreml_model.get_spec().description.input[0]).split('\n')
        name_1 = split_[0].replace('name: "', '')
        name_1 = name_1.replace('"', '')

        size3 = os.path.getsize(coreml_path+framework+'/{}/{}.mlmodel'.format(model_short_name, model_name))

        since_1 = time.time()
        output_dict_test = coreml_model.predict({name_1:x})
        t_elapsed_6 = time.time() - since_1
        t_elapsed2_ = time.time() - since_

        abs_err2 = np.absolute(k_predict-output_dict_test['Identity'])
        rel_err2 = np.absolute(k_predict-output_dict_test['Identity'])/ np.absolute(output_dict_test['Identity'])

        for j in range (len(abs_err)):
            for k in range(len(abs_err[j])): 
                data_writer_error.writerow([model_short_name,framework, training_id, model_name, batch_size, i, abs_err[j][k], rel_err[j][k]])


        for j in range (len(abs_err2)):
            for k in range(len(abs_err2[j])): 
                data_writer_error2.writerow([model_short_name,framework, training_id, model_name, batch_size, i, abs_err2[j][k], rel_err2[j][k]])

        data_writer_run.writerow([model_short_name, framework, training_id, model_name, batch_size, i,'onnx',size0, size2, t_elapsed_1, t_elapsed_3,  t_elapsed_2,'', t_elapsed_, np.mean(abs_err), np.median(abs_err), np.min(abs_err), np.max(abs_err), np.mean(rel_err), np.median(rel_err), np.min(rel_err), np.max(rel_err)])
        data_writer_run.writerow([model_short_name, framework, training_id, model_name, batch_size, i,'coremltools',size0, size3, t_elapsed_1, t_elapsed_6, t_elapsed_4, t_elapsed_5, (t_elapsed2_-t_elapsed_onnx), np.mean(abs_err2), np.median(abs_err2), np.min(abs_err2), np.max(abs_err2), np.mean(rel_err2), np.median(rel_err2), np.min(rel_err2), np.max(rel_err2)])
    except:
        print('    ------ Error during the conversion -----')

In [16]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

def _lets_convert(data, data_writer_error, data_writer_error2, data_writer_run, batch_size):
    since = time.time()
    for i, (inputs, labels) in enumerate(data):
        inputs = to_numpy(inputs)
        labels = to_numpy(labels)
        to_onnx(i, inputs,data_writer_error, data_writer_error2, data_writer_run, batch_size)
        if i == 10:
            break
    time_elapsed = time.time() - since
    print('Conversion complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60) )
    
    #return list_converted
    #return list_converted, abs_errors, rel_errors, 'Conversion complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60), np.mean(np.array(t0_list)), np.mean(np.array(t1_list)), np.mean(np.array(t2_list)), np.mean(np.array(t3_list)), np.mean(np.array(s_list))

In [17]:
#model_name = 'letnet5-keras'
import pandas as pd 
def run_experiment(model_short_name, model_name):
    if not os.path.exists(error_path+framework+"/{}".format(model_short_name)): 
        os.makedirs(error_path+framework+"/{}".format(model_short_name))

    data_file_error = open(error_path + framework+'/{}/onnx_error_metrics_{}.csv'.format(model_short_name, model_name), mode='w', newline='',
                                      encoding='utf-8')
    data_writer_error = csv.writer(data_file_error, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    data_writer_error.writerow(['model','framework', 'training_id', 'model_full', "batch_size", 'round','absolute_error', 'relative_error'])


    data_file_error2 = open(error_path + framework+'/{}/mlmore_error_metrics_{}.csv'.format(model_short_name, model_name), mode='w', newline='',
                                      encoding='utf-8')
    data_writer_error2 = csv.writer(data_file_error2, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    data_writer_error2.writerow(['model','framework', 'training_id', 'model_full', "batch_size", 'round','absolute_error', 'relative_error'])


    data_file_run = open(error_path + framework+'/{}/runtime_metrics_{}.csv'.format(model_short_name, model_name), mode='w', newline='',
                                      encoding='utf-8')
    data_writer_run = csv.writer(data_file_run, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    data_writer_run.writerow(['model','framework', 'training_id', 'model_full', "batch_size", 'round','converter','original_size', 'converted_size', 'original_infererence_time', 'converted_infererence_time', 'conversion_time', 'saving_converted_time', 'overral_time', 'avg_abs_error', 'median_abs_error', 'min_abs_error', 'max_abs_error','avg_rel_error', 'median_rel_error', 'min_rel_error', 'max_rel_error'])

    for batch_size in [1, 5,10,50, 100,128, 150,200, 250,300]: # ,350, 400, 450, 500
        print("################ Batch size: ", batch_size)
        # create Tensor datasets
        train_data = TensorDataset(torch.as_tensor(np.array(x_train).astype('int32')), torch.as_tensor(np.array(y_train).astype('int32')))
        valid_data = TensorDataset(torch.as_tensor(np.array(x_test).astype('int32')), torch.as_tensor(np.array(y_test).astype('int32')))

        # dataloaders
        #batch_size = batch_size_

        # make sure to SHUFFLE your data
        train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
        valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
        
        _lets_convert(valid_loader, data_writer_error, data_writer_error2, data_writer_run, batch_size)
        
    data_file_error.close()
    data_file_error2.close()
    data_file_run.close()

In [18]:
run_experiment(model_short_name, model_name)

################ Batch size:  1
converting for batch:  0

*********




Converting Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 6372.25 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 40/40 [00:00<00:00, 1764.52 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 6230.27 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 6073.67 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 40/40 [00:00<00:00, 2141.56 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 10786.23 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 40/40 [00:00<00:00, 3123.55 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 10431.74 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 40/40 [00:00<00:00, 2149.16 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 7755.94 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 40/40 [00:00<00:00, 823.04 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 14/

converting for batch:  1

*********




Converting Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 10888.24 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 40/40 [00:00<00:00, 2821.31 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 4201.51 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 5386.19 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 40/40 [00:00<00:00, 1439.52 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 11208.29 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 40/40 [00:00<00:00, 1743.16 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 4794.66 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 40/40 [00:00<00:00, 2390.09 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 9021.39 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 40/40 [00:00<00:00, 1713.41 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 14

converting for batch:  2

*********




...

converting for batch:  9

*********




Converting Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 8393.40 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 40/40 [00:00<00:00, 1289.32 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 9962.72 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 8848.74 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 40/40 [00:00<00:00, 1615.96 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 9033.89 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 40/40 [00:00<00:00, 2385.33 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 6286.97 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 40/40 [00:00<00:00, 1004.74 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 8148.80 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 40/40 [00:00<00:00, 1377.13 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 14/1

converting for batch:  10

*********




Converting Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 11155.06 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 40/40 [00:00<00:00, 1846.98 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 9179.34 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 14702.12 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 40/40 [00:00<00:00, 2082.78 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 19418.07 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 40/40 [00:00<00:00, 2496.28 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 8569.80 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 40/40 [00:00<00:00, 1719.91 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 9102.50 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 40/40 [00:00<00:00, 1616.08 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 1

Conversion complete in 7m 9s


# Export GRU

In [13]:
model_short_name = 'gru'

In [14]:
path = '/Volumes/Cisco/Fall2021/onnx-exchange/Training/Keras/gru/'
since_0 = time.time()
#model_path = 'tf_Lenet5_mnist_2021-08-24-10:35:35'
model_name = 'tf_gru-imdb_2021-10-30_{}'.format(training_id)
model = tf.keras.models.load_model(path+ model_name+'.h5')
t_elapsed_0 = time.time() - since_0
size0 = os.path.getsize(path+ model_name+'.h5')
size0

239762504

In [15]:
size1 = os.path.getsize(path+ model_name+'.h5')
size1

size2 = os.path.getsize(path+ model_name+'.h5')
print(size1, size2)

239762504 239762504


In [16]:
def to_onnx(i, x, data_writer_error, data_writer_error2, data_writer_run, batch_size):
    # Input to the model
    #device_reset = cuda.get_current_device()
    #device_reset.reset()
    #x.cuda()
   
    print("converting for batch: ", i)
    
    #torch.random.manual_seed(42)
    #x = torch.randn(10000, 3, 32, 32, requires_grad=True)
    since_ = time.time()
    since_1 = time.time()
    #model = torch.load(path+model_name+'.pth')
    try:
        with tf.device('/cpu:0'): 
            k_predict = model.predict(x)
        t_elapsed_1 = time.time() - since_1
        # Export the model

        if not os.path.exists(onnx_path+framework+"/{}".format(model_short_name)):
            os.makedirs(onnx_path+framework+"/{}".format(model_short_name))
        if not os.path.exists(coreml_path+framework+"/{}".format(model_short_name)):
            os.makedirs(coreml_path+framework+"/{}".format(model_short_name))

        since_1 = time.time()

        since_onnx = time.time()
        model_proto, external_tensor_storage = tf2onnx.convert.from_keras(model,
                    input_signature=None, opset=11, custom_ops=None,
                    custom_op_handlers=None, custom_rewriter=None,
                    inputs_as_nchw=None, extra_opset=None, shape_override=None,
                     target=None, large_model=False, output_path=onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))

        t_elapsed_2 = time.time() - since_1


        onnx_model = onnx.load(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
        onnx.checker.check_model(onnx_model)
        size2 = os.path.getsize(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
        #def to_numpy(tensor):
        #    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()
        ort_session = onnxruntime.InferenceSession(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
        since_1 = time.time()
        ort_inputs = {ort_session.get_inputs()[0].name: x}
        ort_outs = ort_session.run(None, ort_inputs)
        t_elapsed_3 = time.time() - since_1
        t_elapsed_ = time.time() - since_
        t_elapsed_onnx = time.time() - since_onnx
        # compare ONNX Runtime and PyTorch results
        print("\n*********\n\n")
        #time_diff = t_elapsed_0+t_elapsed_1, t_elapsed_2, t_elapsed_3

        abs_err = np.absolute(k_predict-ort_outs[0])
        rel_err = np.absolute(k_predict-ort_outs[0])/ np.absolute(ort_outs[0])
        #print('Batch: ', i, abs_err, rel_err)



        ## Converting the coremltool
        since_1 = time.time()
        coreml_model = coremltools.convert(model)
        t_elapsed_4 = time.time() - since_1
        since_1 = time.time()
        coreml_model.save(coreml_path+framework+'/{}/{}.mlmodel'.format(model_short_name, model_name))
        t_elapsed_5 = time.time() - since_1

        split_ = str(coreml_model.get_spec().description.input[0]).split('\n')
        name_1 = split_[0].replace('name: "', '')
        name_1 = name_1.replace('"', '')

        size3 = os.path.getsize(coreml_path+framework+'/{}/{}.mlmodel'.format(model_short_name, model_name))

        since_1 = time.time()
        output_dict_test = coreml_model.predict({name_1:x})
        t_elapsed_6 = time.time() - since_1
        t_elapsed2_ = time.time() - since_

        abs_err2 = np.absolute(k_predict-output_dict_test['Identity'])
        rel_err2 = np.absolute(k_predict-output_dict_test['Identity'])/ np.absolute(output_dict_test['Identity'])

        for j in range (len(abs_err)):
            for k in range(len(abs_err[j])): 
                data_writer_error.writerow([model_short_name,framework, training_id, model_name, batch_size, i, abs_err[j][k], rel_err[j][k]])


        for j in range (len(abs_err2)):
            for k in range(len(abs_err2[j])): 
                data_writer_error2.writerow([model_short_name,framework, training_id, model_name, batch_size, i, abs_err2[j][k], rel_err2[j][k]])

        data_writer_run.writerow([model_short_name, framework, training_id, model_name, batch_size, i,'onnx',size0, size2, t_elapsed_1, t_elapsed_3,  t_elapsed_2,'', t_elapsed_, np.mean(abs_err), np.median(abs_err), np.min(abs_err), np.max(abs_err), np.mean(rel_err), np.median(rel_err), np.min(rel_err), np.max(rel_err)])
        data_writer_run.writerow([model_short_name, framework, training_id, model_name, batch_size, i,'coremltools',size0, size3, t_elapsed_1, t_elapsed_6, t_elapsed_4, t_elapsed_5, (t_elapsed2_-t_elapsed_onnx), np.mean(abs_err2), np.median(abs_err2), np.min(abs_err2), np.max(abs_err2), np.mean(rel_err2), np.median(rel_err2), np.min(rel_err2), np.max(rel_err2)])
        
    except:
        print(' -------- Error during the conversion -------')

In [17]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

def _lets_convert(data, data_writer_error, data_writer_error2, data_writer_run, batch_size):
    since = time.time()
    for i, (inputs, labels) in enumerate(data):
        inputs = to_numpy(inputs)
        labels = to_numpy(labels)
        to_onnx(i, inputs,data_writer_error, data_writer_error2, data_writer_run, batch_size)
        if i == 10:
            break
    time_elapsed = time.time() - since
    print('Conversion complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60) )
    
    #return list_converted
    #return list_converted, abs_errors, rel_errors, 'Conversion complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60), np.mean(np.array(t0_list)), np.mean(np.array(t1_list)), np.mean(np.array(t2_list)), np.mean(np.array(t3_list)), np.mean(np.array(s_list))

In [18]:
import pandas as pd 
def run_experiment(model_short_name, model_name):
    if not os.path.exists(error_path+framework+"/{}".format(model_short_name)): 
        os.makedirs(error_path+framework+"/{}".format(model_short_name))

    data_file_error = open(error_path + framework+'/{}/onnx_error_metrics_{}.csv'.format(model_short_name, model_name), mode='w', newline='',
                                      encoding='utf-8')
    data_writer_error = csv.writer(data_file_error, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    data_writer_error.writerow(['model','framework', 'training_id', 'model_full', "batch_size", 'round','absolute_error', 'relative_error'])


    data_file_error2 = open(error_path + framework+'/{}/mlmore_error_metrics_{}.csv'.format(model_short_name, model_name), mode='w', newline='',
                                      encoding='utf-8')
    data_writer_error2 = csv.writer(data_file_error2, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    data_writer_error2.writerow(['model','framework', 'training_id', 'model_full', "batch_size", 'round','absolute_error', 'relative_error'])


    data_file_run = open(error_path + framework+'/{}/runtime_metrics_{}.csv'.format(model_short_name, model_name), mode='w', newline='',
                                      encoding='utf-8')
    data_writer_run = csv.writer(data_file_run, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    data_writer_run.writerow(['model','framework', 'training_id', 'model_full', "batch_size", 'round','converter','original_size', 'converted_size', 'original_infererence_time', 'converted_infererence_time', 'conversion_time', 'saving_converted_time', 'overral_time', 'avg_abs_error', 'median_abs_error', 'min_abs_error', 'max_abs_error','avg_rel_error', 'median_rel_error', 'min_rel_error', 'max_rel_error'])

    for batch_size in [1, 5,10,50,100,128, 150,200, 250,300]: # ,350, 400, 450, 500
        print("################ Batch size: ", batch_size)
        # create Tensor datasets
        #train_data = TensorDataset(torch.as_tensor(np.array(x_train).astype('int32')), torch.as_tensor(np.array(y_train).astype('int32')))
        valid_data = TensorDataset(torch.as_tensor(np.array(x_test).astype('int32')), torch.as_tensor(np.array(y_test).astype('int32')))

        # dataloaders
        #batch_size = batch_size_

        # make sure to SHUFFLE your data
        #train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
        valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
        
        _lets_convert(valid_loader, data_writer_error, data_writer_error2, data_writer_run, batch_size)
        
    data_file_error.close()
    data_file_error2.close()
    data_file_run.close()

In [19]:
run_experiment(model_short_name, model_name)

################ Batch size:  1
converting for batch:  0

*********




/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:54: RuntimeWarning: divide by zero encountered in true_divide
Converting Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 19071.21 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 46/46 [00:00<00:00, 2528.61 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 7755.94 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 7204.94 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 46/46 [00:00<00:00, 914.52 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 5116.79 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 46/46 [00:00<00:00, 1268.02 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 3550.63 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 46/46 [00:00<00:00, 1352.26 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 4526.34 ops/s]

Converting Fr

converting for batch:  1

*********




Converting Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 18281.52 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 46/46 [00:00<00:00, 2419.65 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 8058.22 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 7116.74 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 46/46 [00:00<00:00, 1485.98 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 7141.85 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 46/46 [00:00<00:00, 1234.87 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 5133.79 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 46/46 [00:00<00:00, 1211.36 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 9025.55 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 46/46 [00:00<00:00, 1940.03 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 14/

converting for batch:  2

*********




...

converting for batch:  7

*********




Converting Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 19945.74 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 46/46 [00:00<00:00, 3300.12 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 12622.58 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 23182.10 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 46/46 [00:00<00:00, 4086.46 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 13526.90 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 46/46 [00:00<00:00, 2850.40 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 16550.24 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 46/46 [00:00<00:00, 3301.36 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 15874.63 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 46/46 [00:00<00:00, 3733.75 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████

converting for batch:  8
 -------- Error during the conversion -------
converting for batch:  9

*********




Converting Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 13231.24 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 46/46 [00:00<00:00, 3728.05 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 20082.17 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 21152.83 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 46/46 [00:00<00:00, 3470.17 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 17676.18 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 46/46 [00:00<00:00, 4291.90 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 18419.15 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 46/46 [00:00<00:00, 3348.63 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 15675.46 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 46/46 [00:00<00:00, 2997.93 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████

converting for batch:  10
 -------- Error during the conversion -------
Conversion complete in 2m 8s
